In [ ]:
# import the necessary packages
from keras import backend as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense

from tensorflow.keras.layers import BatchNormalization

from keras.datasets import mnist
from keras.utils import np_utils
from tensorflow.keras.optimizers import SGD, RMSprop, Adam

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math

np.random.seed(5)

In [ ]:
def generate_data(N, M, n_symbol):
    a = np.array(range(pow(2, M)))
    K = np.tile(a, n_symbol)
    data = K

    a = np.eye(modulation_order)
    K = np.tile(a, n_symbol)
    label = np.transpose(K)
    return data, label


def generate(M, N, n_symbol):
    #Generate BPSK
    data,label = generate_data(N, M, n_symbol)

    ran1 = np.random.permutation(n_symbol * pow(2, M))
    ran2 = np.random.permutation(n_symbol * pow(2, M))

    symbol1 = 2 * data[ran1] - 1
    symbol2 = 2 * data[ran2] - 1

    SPC = math.sqrt(0.8) * symbol1 + math.sqrt(0.2) * symbol2

    label1 = label[ran1,:].astype('float32')
    label2 = label[ran2,:].astype('float32')

    return SPC, label1, label2


def generate_input(H1_real, H1_image, SPC, N, n_symbol, sigma):
    #add fading channel and AWGN
    N_real, N_image = generate_channel(N, n_symbol * pow(2, N), 0)

    input1_real = H1_real * SPC + sigma * N_real
    input1_img = H1_image * SPC + sigma * N_image

    input1 = np.transpose(np.concatenate((input1_real, input1_img), axis=0))

    return input1


def generate_channel(N, M, k):
    h1 = np.random.randn(N, M) / math.sqrt(2)
    h2 = np.random.randn(N, M) / math.sqrt(2)

    if k == 0:
        return h1, h2
    else:
        return 2 * h1, 2 * h2


In [ ]:
#define the DNN 7 layers
def model_dnn():
  model = Sequential()
  # dense 1 => RELU layers
  model.add(BatchNormalization(scale=True,center=True))
  model.add(Dense(2* modulation_order,kernel_initializer=tf.initializers.truncated_normal(stddev=0.01)))
  model.add(Activation("relu"))

  # dense 2 => RELU layers
  model.add(BatchNormalization(scale=True,center=True))
  model.add(Dense(2* modulation_order,kernel_initializer=tf.initializers.truncated_normal(stddev=0.01)))
  model.add(Activation("relu"))

  # dense 3 => RELU layers
  model.add(BatchNormalization(scale=True,center=True))
  model.add(Dense(2* modulation_order,kernel_initializer=tf.initializers.truncated_normal(stddev=0.01)))
  model.add(Activation("relu"))

  # dense 4 => RELU layers
  model.add(BatchNormalization(scale=True,center=True))
  model.add(Dense(2 * modulation_order,kernel_initializer=tf.initializers.truncated_normal(stddev=0.01)))
  model.add(Activation("relu"))

  # dense 5 => RELU layers
  model.add(BatchNormalization(scale=True,center=True))
  model.add(Dense(2 * modulation_order,kernel_initializer=tf.initializers.truncated_normal(stddev=0.01)))
  model.add(Activation("relu"))

  # dense 6 => RELU layers
  model.add(Dense(modulation_order))
  model.add(Activation("softmax"))

  return model


In [ ]:
def my_loss(y_t , y_p) :
    m_loss = tf.reduce_mean(tf.compat.v2.nn.softmax_cross_entropy_with_logits(logits=y_p, labels=y_t))
    return m_loss

def my_acc(y_t, y_p):
    #K.print_tensor(y_t)
    #K.print_tensor(y_p)
    correct_prediction = tf.equal(tf.argmax(y_p, axis= 1), tf.argmax(y_t, axis= 1))
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return acc

In [ ]:
# network and training
VERBOSE = "auto"
OPTIMIZER = Adam(epsilon= 5e-04)
VALIDATION_SPLIT=0.3
N = 1
M = 1
n_symbol = 100000
n_iteration = 20
NB_EPOCH = 50
modulation_order = 2

SNR_db = np.array(list(range(0, 21, 2)))

batch_size = 100
test_size = 0.5

ERROR_user1 = np.zeros([len(SNR_db), n_iteration])
ERROR_user2 = np.zeros([len(SNR_db), n_iteration])

In [ ]:
for k in range(n_iteration):

    print('training iteration %d' % (k))
    H1_real, H1_image = generate_channel(N, M, 0)
    H2_real, H2_image = generate_channel(N, M, 1)

    for i in range(len(SNR_db)):

        #user 1
        print('User1 , SNR : {}'.format(SNR_db[i]))
        SPC1, label1, _ = generate(M, N, n_symbol)
        signal_power1 = np.mean(pow(SPC1, 2))
        sigma_user1 = math.sqrt(signal_power1 / (math.sqrt(N) * pow(10, float(SNR_db[i]) / 10.0)))
        input1_train = generate_input(H1_real, H1_image, SPC1, N, n_symbol, sigma_user1)


        # initialize the optimizer and model
        model1 = model_dnn()
        model1.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,metrics=[my_acc])
        history1 = model1.fit(input1_train, label1, epochs=NB_EPOCH,verbose=VERBOSE ,validation_split=VALIDATION_SPLIT ,batch_size= batch_size )

        #user 2
        print('User2 , SNR : {}'.format(SNR_db[i]))
        SPC2, _, label2 = generate(M, N, n_symbol)
        signal_power2 = np.mean(pow(SPC2, 2))
        sigma_user2 = math.sqrt(signal_power2 / (math.sqrt(N) * pow(10, float(SNR_db[i]) / 10.0)))
        input2_train = generate_input(H2_real, H2_image, SPC2, N, n_symbol, sigma_user2)

        # initialize the optimizer and model
        model2 = model_dnn()
        model2.compile(loss= "categorical_crossentropy" , optimizer=OPTIMIZER,metrics= [my_acc] )
        history2 = model2.fit(input2_train, label2, epochs=NB_EPOCH,verbose=VERBOSE ,validation_split=VALIDATION_SPLIT , batch_size= batch_size )

        #testing
        print('testing operation %d'%(k))

        SPC_test, test_label1, test_label2 = generate(M, N, int(n_symbol * test_size ) )
        signal_power = np.mean(pow(SPC_test, 2))
        sigma = math.sqrt(signal_power / (math.sqrt(N) * pow(10, SNR_db[i] / 10)))

        input1_test = generate_input(H1_real, H1_image, SPC_test, N, int(n_symbol * test_size), sigma)
        score1 = model1.evaluate(input1_test,test_label1, verbose=VERBOSE)
        ERROR_user1[i,k] = 1-score1[1]


        input2_test = generate_input(H2_real, H2_image, SPC_test, N, int(n_symbol  * test_size), sigma)
        score2 = model2.evaluate(input2_test,test_label2, verbose=VERBOSE)
        ERROR_user2[i,k] = 1-score2[1]


error1 = np.mean((ERROR_user1),axis=1)
error2 = np.mean((ERROR_user2),axis=1)


training iteration 0
User1 , SNR : 0
Epoch 1/50
1400/1400 [==============================] - 10s 5ms/step - loss: 0.6072 - my_acc: 0.6790 - val_loss: 0.6021 - val_my_acc: 0.6805
Epoch 2/50
1400/1400 [==============================] - 6s 4ms/step - loss: 0.5919 - my_acc: 0.6829 - val_loss: 0.5935 - val_my_acc: 0.6847
Epoch 3/50
1400/1400 [==============================] - 6s 4ms/step - loss: 0.5912 - my_acc: 0.6828 - val_loss: 0.5954 - val_my_acc: 0.6814
Epoch 4/50
1400/1400 [==============================] - 5s 3ms/step - loss: 0.5909 - my_acc: 0.6824 - val_loss: 0.5907 - val_my_acc: 0.6831
Epoch 5/50
1400/1400 [==============================] - 5s 3ms/step - loss: 0.5912 - my_acc: 0.6838 - val_loss: 0.5898 - val_my_acc: 0.6833
Epoch 6/50
1400/1400 [==============================] - 5s 3ms/step - loss: 0.5911 - my_acc: 0.6828 - val_loss: 0.5970 - val_my_acc: 0.6806
Epoch 7/50
1400/1400 [==============================] - 5s 3ms/step - loss: 0.5907 - my_acc: 0.6844 - val_loss: 0.5925 - v

In [ ]:
plt.figure()
plt.semilogy(SNR_db,error1, ls = '--', marker = 'o',label='user1')
plt.semilogy(SNR_db,error2, ls = '--', marker = '+',label='user2')
plt.grid()
plt.legend()
plt.ylim(pow(10,-6),pow(10,0) )
plt.xlabel('SNR')
plt.ylabel('SER')
plt.title('SER of 2-users in SISO_NOMA BPSK_DNN')
plt.savefig('res1')
plt.show()

print(error1)
print(error2)

In [ ]:
# list all data in history
# summarize history for accuracy
plt.figure()
plt.plot(history1.history['my_acc'])
plt.plot(history1.history['val_my_acc'])
plt.title('model-1 accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train UE1', 'test UE1'], loc='upper right')
plt.show()

In [ ]:
# list all data in history
# summarize history for accuracy
plt.figure()
plt.plot(history2.history['my_acc'])
plt.plot(history2.history['val_my_acc'])
plt.title('model-2 accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train UE2', 'test UE2'], loc='upper right')
plt.show()

In [ ]:
# summarize history for loss
plt.figure()
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train UE1', 'test UE1'], loc='upper right')
plt.show()

In [ ]:
# summarize history for loss
plt.figure()
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train UE2', 'test UE2'], loc='upper right')
plt.show()

In [ ]:
print(input1_train.shape)
print(input2_train.shape)
print(input1_test.shape)
print(input2_test.shape)

print(label1.shape)
print(label2.shape)
print(test_label1.shape)
print(test_label2.shape)

print(input1_train[11])
print(label1[11])

In [ ]:
model1.save("model_dnn_UE1")
model2.save("mode2_dnn_UE2")